In [1]:
import wfdb
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from python_speech_features.base import mfcc
import scipy.io.wavfile as wav
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import GRU
import numpy as np
from scipy.ndimage import zoom
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
import tensorflow as tf
import pywt
from gen_wavelet_images import generate_wavelet_images as gen_wav
from scipy import signal

In [2]:
import concurrent.futures
import multiprocessing as mp
mp.set_start_method("spawn")

In [3]:
# RESAMPLE
FRAME_TIME = 3.5 # The frame time (in seconds) to divide the signals into equally sized samples
FRAME_OVERLAP = 0 # The frame overlap (in seconds)
RESAMPLE_FACTOR = 1
TRAIN_SIZE = 0.7
fr = 2000

## Data Loading and preprocessing

Loading signals, info and resampling

In [4]:
# Reads Circor digiscope database files
# .hea -> file header
# .wav -> sound files
def read_files(folder = '', extension = ".wav"):
    cont = 0
    signal_dict_list = []
    for filename in os.listdir(folder):
        
        if filename.endswith(".hea") : 
            record_name = filename[:-4]
            record = wfdb.rdheader(os.path.join(folder, record_name))
            signal = wfdb.rdrecord(os.path.join(folder, record_name)).__dict__['p_signal'][:][::RESAMPLE_FACTOR]
            #plt.plot (signal)
            #plt.show()
            #print (record.__dict__)
            #print (signal)
            fs = record.__dict__['fs']
            cont +=1
            #print(record.__dict__)
            #patient_id = record_name.split('_')[0]
            #signal_dict = {'patient_id': patient_id, 'record_name':record_name, 'fs': fs, 'sig_len':record.__dict__['sig_len'], 'signal': signal}
            signal_dict = { 'record_name':record_name, 'fs': fs, 'sig_len':record.__dict__['sig_len'], 'signal': signal}
           
            signal_dict_list.append(signal_dict)
            
            
            
    return pd.DataFrame(signal_dict_list)

In [5]:
df = read_files(folder=r'C:\Users\helde\Downloads\classification-of-heart-sound-recordings-the-physionet-computing-in-cardiology-challenge-2016-1.0.0\training-bcdef')

In [6]:
df

,record_name,fs,sig_len,signal
0,b0001,2000,16000,"[[54.0], [278.0], [501.0], [729.0], [821.0], [..."
1,b0002,2000,16000,"[[-151.0], [-697.0], [-1210.0], [-1704.0], [-2..."
2,b0003,2000,16000,"[[-19.0], [3.0], [139.0], [171.0], [328.0], [4..."
3,b0004,2000,16000,"[[-12.0], [-42.0], [-46.0], [-79.0], [-326.0],..."
4,b0005,2000,16000,"[[-48.0], [-127.0], [-419.0], [-797.0], [-876...."
...,...,...,...,...
2826,f0110,2000,61344,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
2827,f0111,2000,58752,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
2828,f0112,2000,60864,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
2829,f0113,2000,65952,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."


### Loading patient data

Patient data is loaded, since there are subjects with several recordings (in different campaigns), the train_test_split will ensure that the same patient data doesnt end up in both training and testing sets (making them **mutually exclusive**)

In [7]:
folder = r"C:\Users\helde\Downloads\classification-of-heart-sound-recordings-the-physionet-computing-in-cardiology-challenge-2016-1.0.0"

database_info = pd.read_csv(os.path.join(folder, 'Online_Appendix_training_set.csv'))



In [8]:
database_info.head()

,Challenge record name,Database,Original record name,Diagnosis,Class (-1=normal 1=abnormal),# Beat (automated algorithm),# Beats requiring hand correction,Gender,Age (year),Height (m),...,# Recording in each subject,Transducer site on body,Recording state,Murmurs (2=None 3=Weak 4=Strong 6=Unclear),"Murmur Location (2=Sys,3=Dia,4=Both)",Arrhythmia (2=None 3=One arrhythmic beat 4=Medium Arrhytmia 5=Severe Arrhytmia 6=Unclear 7=Two arrhythmic beats),Respiration noise (2=None 3=Insignificant 4=Weak Resp. Noise<2 s 5=Weak Resp. Noise<3 s 6=Weak Resp. Noise <4 s 7=Weak Resp. Noise >4 s 8=Powerfull Resp. Noise<2 s 9=Powerfull Resp. Noise<3 s 10=Powerfull Resp. noise <4 s 11=Powerfull Resp. Noise >4 s 12=Dont know 13=Weak Resp. Noise<1 s 14=Resp. Noise<1 s 15=Powerfull Resp. noise <1 s 16=Weak Rec. Noise<1 s 17=Powerfull Rec. Noise<1 s),Ambient noise (2=None 3=Insignificant 4=Weak Rec. BG<2 s 5=Weak Rec. BG<3 s 6=Weak Rec. BG <4 s 7=Weak Rec. BG >4 s 8=BG. Noise<2 s 9=BG. Noise<3 s 10= BG. noise <4 s 11=BG. Noise >4 s 12=Powerfull BG. Noise<2 s 13=Powerfull BG. Noise<3 s 14=Powerfull BG. noise <4 s 15=Powerfull BG. Noise >4 s 16=Dont know 17=Weak BG. Noise<1 s 18=BG.Noise<1 s 19=Powerfull BG. Noise<1 s),Recording noise (2=None 3=Insignificant 4=Beg. Rec. Noise 5=End Rec. Noise 6=Beg-End Reco Noise 7=Weak Rec. Noise<2 s 8=Weak Rec. Noise<3 s 9=Weak Rec. Noise <4 s 10=Weak Rec. Noise >4 s 11=Powerfull Rec. Noise<2 s 12=Powerfull Rec. Noise<3 s 13=Powerfull Rec. noise <4 s 14=Powerfull Rec. Noise >4 s 15=Spikes 16=Dont know 17=Weak Rec. Noise<1 s 18=Powerfull Rec. Noise<1 s),Abdominal sounds (2=None 3=Insignificant 4=Sto. Noise<2 s 5=Sto. Noise<3 s 6=Sto. Noise <4 s 7=Sto. Noise >4 s 8=Dont know 9=Sto. Noise<1 s)
0,a0001,training-a,C45S1,MVP,1,36,0,NaN,NaN,NaN,...,NaN,Left of parasternum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a0002,training-a,C19S3,MVP,1,27,7,NaN,NaN,NaN,...,NaN,Parasternum when sit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a0003,training-a,C23S0,MVP,1,25,0,NaN,NaN,NaN,...,NaN,Left of parasternum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,a0004,training-a,C35S18,Benign,1,35,0,NaN,NaN,NaN,...,NaN,Left of parasternum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a0005,training-a,C63S20,Benign,1,52,0,NaN,NaN,NaN,...,NaN,Parasternum when squat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df2 =database_info.merge(df, left_on='Challenge record name', right_on = 'record_name', how='right')

In [10]:
df2

,Challenge record name,Database,Original record name,Diagnosis,Class (-1=normal 1=abnormal),# Beat (automated algorithm),# Beats requiring hand correction,Gender,Age (year),Height (m),...,"Murmur Location (2=Sys,3=Dia,4=Both)",Arrhythmia (2=None 3=One arrhythmic beat 4=Medium Arrhytmia 5=Severe Arrhytmia 6=Unclear 7=Two arrhythmic beats),Respiration noise (2=None 3=Insignificant 4=Weak Resp. Noise<2 s 5=Weak Resp. Noise<3 s 6=Weak Resp. Noise <4 s 7=Weak Resp. Noise >4 s 8=Powerfull Resp. Noise<2 s 9=Powerfull Resp. Noise<3 s 10=Powerfull Resp. noise <4 s 11=Powerfull Resp. Noise >4 s 12=Dont know 13=Weak Resp. Noise<1 s 14=Resp. Noise<1 s 15=Powerfull Resp. noise <1 s 16=Weak Rec. Noise<1 s 17=Powerfull Rec. Noise<1 s),Ambient noise (2=None 3=Insignificant 4=Weak Rec. BG<2 s 5=Weak Rec. BG<3 s 6=Weak Rec. BG <4 s 7=Weak Rec. BG >4 s 8=BG. Noise<2 s 9=BG. Noise<3 s 10= BG. noise <4 s 11=BG. Noise >4 s 12=Powerfull BG. Noise<2 s 13=Powerfull BG. Noise<3 s 14=Powerfull BG. noise <4 s 15=Powerfull BG. Noise >4 s 16=Dont know 17=Weak BG. Noise<1 s 18=BG.Noise<1 s 19=Powerfull BG. Noise<1 s),Recording noise (2=None 3=Insignificant 4=Beg. Rec. Noise 5=End Rec. Noise 6=Beg-End Reco Noise 7=Weak Rec. Noise<2 s 8=Weak Rec. Noise<3 s 9=Weak Rec. Noise <4 s 10=Weak Rec. Noise >4 s 11=Powerfull Rec. Noise<2 s 12=Powerfull Rec. Noise<3 s 13=Powerfull Rec. noise <4 s 14=Powerfull Rec. Noise >4 s 15=Spikes 16=Dont know 17=Weak Rec. Noise<1 s 18=Powerfull Rec. Noise<1 s),Abdominal sounds (2=None 3=Insignificant 4=Sto. Noise<2 s 5=Sto. Noise<3 s 6=Sto. Noise <4 s 7=Sto. Noise >4 s 8=Dont know 9=Sto. Noise<1 s),record_name,fs,sig_len,signal
0,b0001,training-b,S98f2_data,Normal,-1.0,8.0,0.0,M,NaN,NaN,...,0.0,2.0,3.0,3.0,3.0,3.0,b0001,2000,16000,"[[54.0], [278.0], [501.0], [729.0], [821.0], [..."
1,b0002,training-b,S179f1_data,Normal,-1.0,9.0,0.0,M,NaN,NaN,...,0.0,2.0,10.0,3.0,3.0,3.0,b0002,2000,16000,"[[-151.0], [-697.0], [-1210.0], [-1704.0], [-2..."
2,b0003,training-b,S173f2_data,Normal,-1.0,8.0,0.0,M,NaN,NaN,...,0.0,2.0,9.0,14.0,3.0,3.0,b0003,2000,16000,"[[-19.0], [3.0], [139.0], [171.0], [328.0], [4..."
3,b0004,training-b,S73f5_data,Normal,-1.0,9.0,0.0,M,NaN,NaN,...,0.0,2.0,5.0,4.0,3.0,2.0,b0004,2000,16000,"[[-12.0], [-42.0], [-46.0], [-79.0], [-326.0],..."
4,b0005,training-b,S145f1_data,Normal,-1.0,9.0,0.0,M,NaN,NaN,...,0.0,2.0,3.0,7.0,6.0,2.0,b0005,2000,16000,"[[-48.0], [-127.0], [-419.0], [-797.0], [-876...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2826,f0110,training-f,a110,Normal,-1.0,46.0,0.0,F,27.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,f0110,2000,61344,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
2827,f0111,training-f,a111,Normal,-1.0,51.0,0.0,F,58.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,f0111,2000,58752,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
2828,f0112,training-f,a112,Normal,-1.0,21.0,21.0,M,71.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,f0112,2000,60864,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
2829,f0113,training-f,a68-1,Normal,-1.0,32.0,15.0,M,26.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,f0113,2000,65952,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."


In [11]:
df2['y']=df2['Class (-1=normal 1=abnormal)'].apply(lambda x: 1 if x==1 else 0)

### Train test split

Splits the data in train test, not leaking the same subject on both sets

In [12]:
gss = GroupShuffleSplit(n_splits=1, train_size=TRAIN_SIZE, random_state=0)


In [13]:
X= np.array(df2['signal'])
y=  np.array(df2['y'])
groups= np.array( df2['record_name'])



In [14]:
train_idx, test_idx = next(gss.split(X, y, groups))

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

In [15]:
frame_time = FRAME_TIME # time in seconds desired
frame_overlap = FRAME_OVERLAP # overlap in seconds

frame_length = int (fr * frame_time)
frame_step = int(frame_length - fr * frame_overlap)


In [16]:
X_train.shape

(1981,)

In [17]:
def create_frames ( pcg, y):
    f_y_train = []
    X_pcg = []
    record_id = []
    for index in range (len (pcg)):
        print (index)
        pcg_frames = tf.signal.frame(pcg[index].T, frame_length= frame_length, frame_step = frame_step, pad_end=False).numpy().tolist()[0]
        #print(pcg_frames)
        X_pcg.extend(pcg_frames)
        
        f_y_train = f_y_train + [y[index]] * len (pcg_frames)
        record_id.extend([index]* len (pcg_frames))
        
    return  X_pcg, f_y_train, record_id


In [18]:
X_train_pcg, f_y_train, record_id = create_frames ( X_train, y_train)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980


In [19]:
X_test_pcg, f_y_test, record_id_test = create_frames ( X_test, y_test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [20]:
record_id_test

[0,
 0,
 1,
 1,
 2,
 2,
 3,
 3,
 4,
 4,
 5,
 5,
 6,
 6,
 7,
 7,
 8,
 8,
 9,
 9,
 10,
 10,
 11,
 11,
 12,
 12,
 13,
 13,
 14,
 14,
 15,
 15,
 16,
 16,
 17,
 17,
 18,
 18,
 19,
 19,
 20,
 20,
 21,
 21,
 22,
 22,
 23,
 23,
 24,
 24,
 25,
 25,
 26,
 26,
 27,
 27,
 28,
 28,
 29,
 29,
 30,
 30,
 31,
 31,
 32,
 32,
 33,
 33,
 34,
 34,
 35,
 35,
 36,
 36,
 37,
 37,
 38,
 38,
 39,
 39,
 40,
 40,
 41,
 41,
 42,
 42,
 43,
 43,
 44,
 44,
 45,
 45,
 46,
 46,
 47,
 47,
 48,
 48,
 49,
 49,
 50,
 50,
 51,
 51,
 52,
 52,
 53,
 53,
 54,
 54,
 55,
 55,
 56,
 56,
 57,
 57,
 58,
 58,
 59,
 59,
 60,
 60,
 61,
 61,
 62,
 62,
 63,
 63,
 64,
 64,
 65,
 65,
 66,
 66,
 67,
 67,
 68,
 68,
 69,
 69,
 70,
 70,
 71,
 71,
 72,
 72,
 73,
 73,
 74,
 74,
 75,
 75,
 76,
 76,
 77,
 77,
 78,
 78,
 79,
 79,
 80,
 80,
 81,
 81,
 82,
 82,
 83,
 83,
 84,
 84,
 85,
 85,
 86,
 86,
 87,
 87,
 88,
 88,
 89,
 89,
 90,
 90,
 91,
 91,
 92,
 92,
 93,
 93,
 94,
 94,
 95,
 95,
 96,
 96,
 97,
 97,
 98,
 98,
 99,
 99,
 100,
 100,
 101,
 1

Reshaping

In [21]:


X_test_pcg = np.array(X_test_pcg)
X_train_pcg = np.array(X_train_pcg)

X_test_pcg = np.nan_to_num(X_test_pcg, copy=True, nan=0.0, posinf=None, neginf=None)
X_train_pcg  = np.nan_to_num(X_train_pcg, copy=True, nan=0.0, posinf=None, neginf=None)


X_test_pcg = X_test_pcg.reshape(X_test_pcg.shape[0], X_test_pcg.shape[1], 1)
X_train_pcg = X_train_pcg.reshape(X_train_pcg.shape[0], X_train_pcg.shape[1], 1)



#### Standardization

In [22]:
def standardize_signals(signals):
    standardized_signals = []
    for signal in signals:
        mean = np.mean(signal)
        #print (mean)
        std = np.std(signal)
        standardized_signal = (signal - mean) / std
        standardized_signals.append(standardized_signal)
    return np.array(standardized_signals)

In [32]:
plt.plot(X_train_pcg[1000])
plt.savefig('unfiltered_pcg.png')
plt.close()

In [23]:
X_train_pcg = standardize_signals(X_train_pcg)
X_test_pcg = standardize_signals(X_test_pcg)

<ipython-input-22-5431602d1786>:7: RuntimeWarning: invalid value encountered in divide
  standardized_signal = (signal - mean) / std


### Bandpass filter 



In [24]:
def butter_bandpass(lowcut, highcut, fs, order, signals):
    # Normalized cutoff frequencies
    b, a = signal.butter(order, [lowcut, highcut], btype='band', fs=fs)
    
    filtered_signals = []
    for s in signals:
        s= np.ravel(s)
        #print (s.shape)
        y = signal.filtfilt(b, a, s)
        
        filtered_signals.append(y)
    return np.array(filtered_signals)


In [25]:
lowcut = 25
highcut = 400
fs = 2000
order = 4


X_train_pcg = butter_bandpass(lowcut, highcut, fs, order, X_train_pcg)
X_test_pcg = butter_bandpass(lowcut, highcut, fs, order, X_test_pcg)

In [26]:
X_test_pcg = X_test_pcg.reshape(X_test_pcg.shape[0], X_test_pcg.shape[1], 1)
X_train_pcg = X_train_pcg.reshape(X_train_pcg.shape[0], X_train_pcg.shape[1], 1)

In [27]:
plt.plot(X_test_pcg[728])
plt.savefig('test_pcg.png')
plt.close()

In [28]:

if __name__ == '__main__':
    fs = 2000
    wav = pywt.ContinuousWavelet("morl")
    scales = np.arange(5, 130)
    directory = 'physionet2016_bcdef\\test'
    record_ecg_test_list = []

    with concurrent.futures.ProcessPoolExecutor(12) as executor:
            futures = [executor.submit(gen_wav, sig, r ,y, fs, wav, scales, directory, i) for sig,r, y,i in  zip(X_test_pcg, record_id_test, f_y_test, range(len(X_test_pcg)))]
            for future in concurrent.futures.as_completed(futures):
                print (future.result())
                record_ecg_test_list.append(future.result())
            

    df = pd.DataFrame(record_ecg_test_list)
    df.to_csv(directory+'\\dataset.csv')   

{'filename': '0_1.tiff', 'label': 0}
{'filename': '1_2.tiff', 'label': 0}
{'filename': '2_4.tiff', 'label': 0}
{'filename': '0_0.tiff', 'label': 0}
{'filename': '3_6.tiff', 'label': 0}
{'filename': '2_5.tiff', 'label': 0}
{'filename': '4_9.tiff', 'label': 0}
{'filename': '1_3.tiff', 'label': 0}
{'filename': '3_7.tiff', 'label': 0}
{'filename': '5_11.tiff', 'label': 1}
{'filename': '4_8.tiff', 'label': 0}
{'filename': '5_10.tiff', 'label': 1}
{'filename': '6_12.tiff', 'label': 1}
{'filename': '6_13.tiff', 'label': 1}
{'filename': '8_17.tiff', 'label': 1}
{'filename': '7_15.tiff', 'label': 0}
{'filename': '9_18.tiff', 'label': 0}
{'filename': '9_19.tiff', 'label': 0}
{'filename': '7_14.tiff', 'label': 0}
{'filename': '8_16.tiff', 'label': 1}
{'filename': '10_20.tiff', 'label': 0}
{'filename': '11_22.tiff', 'label': 0}
{'filename': '10_21.tiff', 'label': 0}
{'filename': '11_23.tiff', 'label': 0}
{'filename': '12_24.tiff', 'label': 0}
{'filename': '13_26.tiff', 'label': 1}
{'filename': '12

{'filename': '104_208.tiff', 'label': 0}
{'filename': '104_209.tiff', 'label': 0}
{'filename': '105_211.tiff', 'label': 0}
{'filename': '105_210.tiff', 'label': 0}
{'filename': '106_212.tiff', 'label': 0}
{'filename': '107_214.tiff', 'label': 0}
{'filename': '107_215.tiff', 'label': 0}
{'filename': '108_216.tiff', 'label': 1}
{'filename': '106_213.tiff', 'label': 0}
{'filename': '108_217.tiff', 'label': 1}
{'filename': '109_218.tiff', 'label': 0}
{'filename': '109_219.tiff', 'label': 0}
{'filename': '110_220.tiff', 'label': 0}
{'filename': '110_221.tiff', 'label': 0}
{'filename': '111_222.tiff', 'label': 0}
{'filename': '111_223.tiff', 'label': 0}
{'filename': '112_224.tiff', 'label': 1}
{'filename': '112_225.tiff', 'label': 1}
{'filename': '113_227.tiff', 'label': 0}
{'filename': '114_228.tiff', 'label': 0}
{'filename': '113_226.tiff', 'label': 0}
{'filename': '114_229.tiff', 'label': 0}
{'filename': '116_232.tiff', 'label': 0}
{'filename': '116_233.tiff', 'label': 0}
{'filename': '11

{'filename': '164_407.tiff', 'label': 1}
{'filename': '164_411.tiff', 'label': 1}
{'filename': '164_413.tiff', 'label': 1}
{'filename': '164_408.tiff', 'label': 1}
{'filename': '164_415.tiff', 'label': 1}
{'filename': '164_412.tiff', 'label': 1}
{'filename': '164_417.tiff', 'label': 1}
{'filename': '164_418.tiff', 'label': 1}
{'filename': '164_414.tiff', 'label': 1}
{'filename': '164_416.tiff', 'label': 1}
{'filename': '165_419.tiff', 'label': 0}
{'filename': '166_422.tiff', 'label': 0}
{'filename': '167_423.tiff', 'label': 0}
{'filename': '165_420.tiff', 'label': 0}
{'filename': '166_421.tiff', 'label': 0}
{'filename': '168_424.tiff', 'label': 1}
{'filename': '168_425.tiff', 'label': 1}
{'filename': '168_426.tiff', 'label': 1}
{'filename': '168_427.tiff', 'label': 1}
{'filename': '169_429.tiff', 'label': 0}
{'filename': '168_428.tiff', 'label': 1}
{'filename': '169_430.tiff', 'label': 0}
{'filename': '169_431.tiff', 'label': 0}
{'filename': '170_432.tiff', 'label': 0}
{'filename': '17

{'filename': '202_611.tiff', 'label': 0}
{'filename': '202_613.tiff', 'label': 0}
{'filename': '203_614.tiff', 'label': 0}
{'filename': '202_612.tiff', 'label': 0}
{'filename': '203_618.tiff', 'label': 0}
{'filename': '203_615.tiff', 'label': 0}
{'filename': '203_617.tiff', 'label': 0}
{'filename': '203_616.tiff', 'label': 0}
{'filename': '204_619.tiff', 'label': 0}
{'filename': '204_620.tiff', 'label': 0}
{'filename': '204_621.tiff', 'label': 0}
{'filename': '204_622.tiff', 'label': 0}
{'filename': '204_623.tiff', 'label': 0}
{'filename': '204_624.tiff', 'label': 0}
{'filename': '204_625.tiff', 'label': 0}
{'filename': '205_626.tiff', 'label': 1}
{'filename': '205_627.tiff', 'label': 1}
{'filename': '205_629.tiff', 'label': 1}
{'filename': '205_628.tiff', 'label': 1}
{'filename': '205_630.tiff', 'label': 1}
{'filename': '205_631.tiff', 'label': 1}
{'filename': '206_632.tiff', 'label': 0}
{'filename': '206_634.tiff', 'label': 0}
{'filename': '206_633.tiff', 'label': 0}
{'filename': '20

{'filename': '235_815.tiff', 'label': 0}
{'filename': '236_816.tiff', 'label': 1}
{'filename': '236_817.tiff', 'label': 1}
{'filename': '236_818.tiff', 'label': 1}
{'filename': '236_820.tiff', 'label': 1}
{'filename': '236_819.tiff', 'label': 1}
{'filename': '236_821.tiff', 'label': 1}
{'filename': '236_822.tiff', 'label': 1}
{'filename': '236_825.tiff', 'label': 1}
{'filename': '236_824.tiff', 'label': 1}
{'filename': '236_823.tiff', 'label': 1}
{'filename': '236_826.tiff', 'label': 1}
{'filename': '237_829.tiff', 'label': 0}
{'filename': '237_827.tiff', 'label': 0}
{'filename': '237_828.tiff', 'label': 0}
{'filename': '237_831.tiff', 'label': 0}
{'filename': '237_830.tiff', 'label': 0}
{'filename': '238_832.tiff', 'label': 0}
{'filename': '238_833.tiff', 'label': 0}
{'filename': '238_834.tiff', 'label': 0}
{'filename': '238_835.tiff', 'label': 0}
{'filename': '238_836.tiff', 'label': 0}
{'filename': '238_837.tiff', 'label': 0}
{'filename': '238_838.tiff', 'label': 0}
{'filename': '23

...

{'filename': '728_3775.tiff', 'label': 0}
{'filename': '728_3777.tiff', 'label': 0}
{'filename': '728_3776.tiff', 'label': 0}
{'filename': '729_3778.tiff', 'label': 0}
{'filename': '730_3780.tiff', 'label': 0}
{'filename': '729_3779.tiff', 'label': 0}
{'filename': '730_3782.tiff', 'label': 0}
{'filename': '730_3781.tiff', 'label': 0}
{'filename': '730_3784.tiff', 'label': 0}
{'filename': '730_3783.tiff', 'label': 0}
{'filename': '731_3785.tiff', 'label': 1}
{'filename': '731_3787.tiff', 'label': 1}
{'filename': '731_3786.tiff', 'label': 1}
{'filename': '731_3788.tiff', 'label': 1}
{'filename': '732_3790.tiff', 'label': 0}
{'filename': '732_3789.tiff', 'label': 0}
{'filename': '732_3791.tiff', 'label': 0}
{'filename': '732_3792.tiff', 'label': 0}
{'filename': '732_3794.tiff', 'label': 0}
{'filename': '732_3793.tiff', 'label': 0}
{'filename': '733_3795.tiff', 'label': 0}
{'filename': '733_3796.tiff', 'label': 0}
{'filename': '733_3798.tiff', 'label': 0}
{'filename': '733_3797.tiff', 'lab

{'filename': '764_3971.tiff', 'label': 0}
{'filename': '764_3972.tiff', 'label': 0}
{'filename': '764_3973.tiff', 'label': 0}
{'filename': '764_3974.tiff', 'label': 0}
{'filename': '765_3976.tiff', 'label': 0}
{'filename': '765_3975.tiff', 'label': 0}
{'filename': '765_3979.tiff', 'label': 0}
{'filename': '765_3978.tiff', 'label': 0}
{'filename': '765_3977.tiff', 'label': 0}
{'filename': '767_3982.tiff', 'label': 0}
{'filename': '766_3980.tiff', 'label': 1}
{'filename': '766_3981.tiff', 'label': 1}
{'filename': '767_3983.tiff', 'label': 0}
{'filename': '767_3984.tiff', 'label': 0}
{'filename': '767_3985.tiff', 'label': 0}
{'filename': '768_3987.tiff', 'label': 0}
{'filename': '768_3988.tiff', 'label': 0}
{'filename': '768_3986.tiff', 'label': 0}
{'filename': '768_3989.tiff', 'label': 0}
{'filename': '769_3991.tiff', 'label': 0}
{'filename': '769_3990.tiff', 'label': 0}
{'filename': '769_3992.tiff', 'label': 0}
{'filename': '769_3993.tiff', 'label': 0}
{'filename': '770_3995.tiff', 'lab

{'filename': '794_4167.tiff', 'label': 0}
{'filename': '794_4169.tiff', 'label': 0}
{'filename': '794_4168.tiff', 'label': 0}
{'filename': '794_4170.tiff', 'label': 0}
{'filename': '795_4171.tiff', 'label': 0}
{'filename': '795_4172.tiff', 'label': 0}
{'filename': '795_4173.tiff', 'label': 0}
{'filename': '795_4174.tiff', 'label': 0}
{'filename': '795_4175.tiff', 'label': 0}
{'filename': '795_4176.tiff', 'label': 0}
{'filename': '795_4177.tiff', 'label': 0}
{'filename': '795_4178.tiff', 'label': 0}
{'filename': '795_4179.tiff', 'label': 0}
{'filename': '796_4181.tiff', 'label': 0}
{'filename': '795_4180.tiff', 'label': 0}
{'filename': '796_4182.tiff', 'label': 0}
{'filename': '797_4183.tiff', 'label': 0}
{'filename': '797_4184.tiff', 'label': 0}
{'filename': '797_4185.tiff', 'label': 0}
{'filename': '797_4186.tiff', 'label': 0}
{'filename': '797_4187.tiff', 'label': 0}
{'filename': '797_4188.tiff', 'label': 0}
{'filename': '797_4189.tiff', 'label': 0}
{'filename': '798_4190.tiff', 'lab

{'filename': '826_4365.tiff', 'label': 0}
{'filename': '826_4364.tiff', 'label': 0}
{'filename': '826_4366.tiff', 'label': 0}
{'filename': '826_4367.tiff', 'label': 0}
{'filename': '826_4368.tiff', 'label': 0}
{'filename': '827_4370.tiff', 'label': 0}
{'filename': '827_4369.tiff', 'label': 0}
{'filename': '827_4371.tiff', 'label': 0}
{'filename': '827_4372.tiff', 'label': 0}
{'filename': '827_4373.tiff', 'label': 0}
{'filename': '827_4374.tiff', 'label': 0}
{'filename': '827_4375.tiff', 'label': 0}
{'filename': '828_4377.tiff', 'label': 0}
{'filename': '827_4376.tiff', 'label': 0}
{'filename': '828_4378.tiff', 'label': 0}
{'filename': '828_4379.tiff', 'label': 0}
{'filename': '828_4380.tiff', 'label': 0}
{'filename': '828_4381.tiff', 'label': 0}
{'filename': '828_4383.tiff', 'label': 0}
{'filename': '828_4382.tiff', 'label': 0}
{'filename': '828_4384.tiff', 'label': 0}
{'filename': '829_4385.tiff', 'label': 0}
{'filename': '829_4387.tiff', 'label': 0}
{'filename': '829_4386.tiff', 'lab

{'filename': '848_4560.tiff', 'label': 0}
{'filename': '849_4561.tiff', 'label': 0}
{'filename': '849_4562.tiff', 'label': 0}
{'filename': '849_4564.tiff', 'label': 0}
{'filename': '849_4563.tiff', 'label': 0}
{'filename': '849_4565.tiff', 'label': 0}
{'filename': '849_4566.tiff', 'label': 0}
{'filename': '849_4567.tiff', 'label': 0}
{'filename': '849_4568.tiff', 'label': 0}


In [29]:

if __name__ == '__main__':
    fs = 2000
    wav = pywt.ContinuousWavelet("morl")
    scales = np.arange(7, 130)
    directory = 'physionet2016_bcdef\\train'
    record_ecg_test_list = []

    with concurrent.futures.ProcessPoolExecutor(12) as executor:
            futures = [executor.submit(gen_wav, sig, r ,y, fs, wav, scales, directory, i) for sig,r, y,i in  zip(X_train_pcg, record_id, f_y_train, range(len(X_train_pcg)))]
            for future in concurrent.futures.as_completed(futures):
                print (future.result())
                record_ecg_test_list.append(future.result())
            

    df = pd.DataFrame(record_ecg_test_list)
    df.to_csv(directory+'\\dataset.csv')   


{'filename': '2_5.tiff', 'label': 0}
{'filename': '0_0.tiff', 'label': 0}
{'filename': '1_2.tiff', 'label': 0}
{'filename': '2_4.tiff', 'label': 0}
{'filename': '0_1.tiff', 'label': 0}
{'filename': '3_7.tiff', 'label': 0}
{'filename': '3_6.tiff', 'label': 0}
{'filename': '5_10.tiff', 'label': 1}
{'filename': '5_11.tiff', 'label': 1}
{'filename': '1_3.tiff', 'label': 0}
{'filename': '4_9.tiff', 'label': 0}
{'filename': '4_8.tiff', 'label': 0}
{'filename': '6_12.tiff', 'label': 0}
{'filename': '7_14.tiff', 'label': 0}
{'filename': '6_13.tiff', 'label': 0}
{'filename': '8_16.tiff', 'label': 1}
{'filename': '10_20.tiff', 'label': 0}
{'filename': '9_18.tiff', 'label': 0}
{'filename': '10_21.tiff', 'label': 0}
{'filename': '8_17.tiff', 'label': 1}
{'filename': '9_19.tiff', 'label': 0}
{'filename': '7_15.tiff', 'label': 0}
{'filename': '11_23.tiff', 'label': 0}
{'filename': '11_22.tiff', 'label': 0}
{'filename': '12_24.tiff', 'label': 0}
{'filename': '12_25.tiff', 'label': 0}
{'filename': '13

{'filename': '107_212.tiff', 'label': 0}
{'filename': '108_214.tiff', 'label': 1}
{'filename': '109_216.tiff', 'label': 0}
{'filename': '109_215.tiff', 'label': 0}
{'filename': '110_217.tiff', 'label': 0}
{'filename': '111_219.tiff', 'label': 1}
{'filename': '110_218.tiff', 'label': 0}
{'filename': '111_220.tiff', 'label': 1}
{'filename': '112_221.tiff', 'label': 0}
{'filename': '113_223.tiff', 'label': 0}
{'filename': '112_222.tiff', 'label': 0}
{'filename': '114_226.tiff', 'label': 0}
{'filename': '113_224.tiff', 'label': 0}
{'filename': '114_225.tiff', 'label': 0}
{'filename': '115_227.tiff', 'label': 1}
{'filename': '115_228.tiff', 'label': 1}
{'filename': '116_229.tiff', 'label': 0}
{'filename': '116_230.tiff', 'label': 0}
{'filename': '117_231.tiff', 'label': 0}
{'filename': '117_232.tiff', 'label': 0}
{'filename': '118_233.tiff', 'label': 0}
{'filename': '118_234.tiff', 'label': 0}
{'filename': '119_235.tiff', 'label': 0}
{'filename': '119_236.tiff', 'label': 0}
{'filename': '12

{'filename': '211_419.tiff', 'label': 0}
{'filename': '215_427.tiff', 'label': 0}
{'filename': '210_417.tiff', 'label': 0}
{'filename': '213_423.tiff', 'label': 0}
{'filename': '211_420.tiff', 'label': 0}
{'filename': '210_418.tiff', 'label': 0}
{'filename': '212_421.tiff', 'label': 1}
{'filename': '215_428.tiff', 'label': 0}
{'filename': '212_422.tiff', 'label': 1}
{'filename': '214_426.tiff', 'label': 0}
{'filename': '213_424.tiff', 'label': 0}
{'filename': '214_425.tiff', 'label': 0}
{'filename': '216_429.tiff', 'label': 0}
{'filename': '217_432.tiff', 'label': 0}
{'filename': '218_433.tiff', 'label': 0}
{'filename': '221_440.tiff', 'label': 0}
{'filename': '220_438.tiff', 'label': 0}
{'filename': '221_439.tiff', 'label': 0}
{'filename': '218_434.tiff', 'label': 0}
{'filename': '217_431.tiff', 'label': 0}
{'filename': '219_436.tiff', 'label': 0}
{'filename': '219_435.tiff', 'label': 0}
{'filename': '216_430.tiff', 'label': 0}
{'filename': '220_437.tiff', 'label': 0}
{'filename': '22

{'filename': '311_618.tiff', 'label': 1}
{'filename': '310_617.tiff', 'label': 0}
{'filename': '311_619.tiff', 'label': 1}
{'filename': '312_621.tiff', 'label': 0}
{'filename': '313_623.tiff', 'label': 0}
{'filename': '313_622.tiff', 'label': 0}
{'filename': '315_626.tiff', 'label': 0}
{'filename': '319_634.tiff', 'label': 0}
{'filename': '318_632.tiff', 'label': 0}
{'filename': '314_624.tiff', 'label': 0}
{'filename': '312_620.tiff', 'label': 0}
{'filename': '314_625.tiff', 'label': 0}
{'filename': '315_627.tiff', 'label': 0}
{'filename': '316_628.tiff', 'label': 0}
{'filename': '317_630.tiff', 'label': 1}
{'filename': '317_631.tiff', 'label': 1}
{'filename': '316_629.tiff', 'label': 0}
{'filename': '318_633.tiff', 'label': 0}
{'filename': '319_635.tiff', 'label': 0}
{'filename': '320_637.tiff', 'label': 1}
{'filename': '323_642.tiff', 'label': 0}
{'filename': '320_636.tiff', 'label': 1}
{'filename': '325_646.tiff', 'label': 0}
{'filename': '321_639.tiff', 'label': 0}
{'filename': '32

{'filename': '355_821.tiff', 'label': 0}
{'filename': '355_819.tiff', 'label': 0}
{'filename': '355_820.tiff', 'label': 0}
{'filename': '355_822.tiff', 'label': 0}
{'filename': '356_823.tiff', 'label': 1}
{'filename': '356_824.tiff', 'label': 1}
{'filename': '356_826.tiff', 'label': 1}
{'filename': '356_825.tiff', 'label': 1}
{'filename': '356_828.tiff', 'label': 1}
{'filename': '356_827.tiff', 'label': 1}
{'filename': '356_830.tiff', 'label': 1}
{'filename': '356_829.tiff', 'label': 1}
{'filename': '356_831.tiff', 'label': 1}
{'filename': '356_832.tiff', 'label': 1}
{'filename': '356_833.tiff', 'label': 1}
{'filename': '356_834.tiff', 'label': 1}
{'filename': '356_835.tiff', 'label': 1}
{'filename': '356_836.tiff', 'label': 1}
{'filename': '356_837.tiff', 'label': 1}
{'filename': '356_838.tiff', 'label': 1}
{'filename': '357_841.tiff', 'label': 1}
{'filename': '356_839.tiff', 'label': 1}
{'filename': '357_840.tiff', 'label': 1}
{'filename': '357_842.tiff', 'label': 1}
{'filename': '35

...

{'filename': '1894_10338.tiff', 'label': 0}
{'filename': '1894_10339.tiff', 'label': 0}
{'filename': '1894_10341.tiff', 'label': 0}
{'filename': '1894_10340.tiff', 'label': 0}
{'filename': '1895_10343.tiff', 'label': 0}
{'filename': '1895_10342.tiff', 'label': 0}
{'filename': '1895_10344.tiff', 'label': 0}
{'filename': '1896_10345.tiff', 'label': 0}
{'filename': '1896_10346.tiff', 'label': 0}
{'filename': '1896_10347.tiff', 'label': 0}
{'filename': '1896_10349.tiff', 'label': 0}
{'filename': '1896_10348.tiff', 'label': 0}
{'filename': '1896_10350.tiff', 'label': 0}
{'filename': '1896_10351.tiff', 'label': 0}
{'filename': '1897_10353.tiff', 'label': 0}
{'filename': '1897_10352.tiff', 'label': 0}
{'filename': '1897_10355.tiff', 'label': 0}
{'filename': '1897_10354.tiff', 'label': 0}
{'filename': '1897_10356.tiff', 'label': 0}
{'filename': '1897_10357.tiff', 'label': 0}
{'filename': '1897_10358.tiff', 'label': 0}
{'filename': '1898_10359.tiff', 'label': 0}
{'filename': '1898_10360.tiff', 

{'filename': '1915_10525.tiff', 'label': 0}
{'filename': '1915_10526.tiff', 'label': 0}
{'filename': '1915_10527.tiff', 'label': 0}
{'filename': '1916_10531.tiff', 'label': 0}
{'filename': '1916_10530.tiff', 'label': 0}
{'filename': '1915_10529.tiff', 'label': 0}
{'filename': '1915_10528.tiff', 'label': 0}
{'filename': '1916_10532.tiff', 'label': 0}
{'filename': '1916_10533.tiff', 'label': 0}
{'filename': '1916_10535.tiff', 'label': 0}
{'filename': '1916_10534.tiff', 'label': 0}
{'filename': '1916_10536.tiff', 'label': 0}
{'filename': '1916_10537.tiff', 'label': 0}
{'filename': '1916_10538.tiff', 'label': 0}
{'filename': '1917_10539.tiff', 'label': 1}
{'filename': '1917_10543.tiff', 'label': 1}
{'filename': '1917_10540.tiff', 'label': 1}
{'filename': '1917_10541.tiff', 'label': 1}
{'filename': '1917_10542.tiff', 'label': 1}
{'filename': '1917_10544.tiff', 'label': 1}
{'filename': '1917_10545.tiff', 'label': 1}
{'filename': '1917_10546.tiff', 'label': 1}
{'filename': '1917_10547.tiff', 

{'filename': '1936_10714.tiff', 'label': 1}
{'filename': '1937_10715.tiff', 'label': 1}
{'filename': '1937_10716.tiff', 'label': 1}
{'filename': '1937_10717.tiff', 'label': 1}
{'filename': '1937_10718.tiff', 'label': 1}
{'filename': '1937_10719.tiff', 'label': 1}
{'filename': '1937_10720.tiff', 'label': 1}
{'filename': '1937_10721.tiff', 'label': 1}
{'filename': '1937_10722.tiff', 'label': 1}
{'filename': '1938_10723.tiff', 'label': 0}
{'filename': '1938_10724.tiff', 'label': 0}
{'filename': '1938_10725.tiff', 'label': 0}
{'filename': '1938_10728.tiff', 'label': 0}
{'filename': '1938_10727.tiff', 'label': 0}
{'filename': '1938_10726.tiff', 'label': 0}
{'filename': '1938_10729.tiff', 'label': 0}
{'filename': '1938_10730.tiff', 'label': 0}
{'filename': '1938_10732.tiff', 'label': 0}
{'filename': '1938_10731.tiff', 'label': 0}
{'filename': '1938_10733.tiff', 'label': 0}
{'filename': '1939_10734.tiff', 'label': 0}
{'filename': '1939_10736.tiff', 'label': 0}
{'filename': '1939_10735.tiff', 

{'filename': '1957_10902.tiff', 'label': 1}
{'filename': '1958_10903.tiff', 'label': 0}
{'filename': '1958_10904.tiff', 'label': 0}
{'filename': '1958_10905.tiff', 'label': 0}
{'filename': '1958_10906.tiff', 'label': 0}
{'filename': '1958_10908.tiff', 'label': 0}
{'filename': '1958_10907.tiff', 'label': 0}
{'filename': '1958_10909.tiff', 'label': 0}
{'filename': '1958_10910.tiff', 'label': 0}
{'filename': '1958_10911.tiff', 'label': 0}
{'filename': '1959_10912.tiff', 'label': 0}
{'filename': '1959_10915.tiff', 'label': 0}
{'filename': '1959_10913.tiff', 'label': 0}
{'filename': '1959_10914.tiff', 'label': 0}
{'filename': '1959_10916.tiff', 'label': 0}
{'filename': '1959_10917.tiff', 'label': 0}
{'filename': '1960_10921.tiff', 'label': 0}
{'filename': '1959_10919.tiff', 'label': 0}
{'filename': '1959_10918.tiff', 'label': 0}
{'filename': '1959_10920.tiff', 'label': 0}
{'filename': '1960_10925.tiff', 'label': 0}
{'filename': '1960_10922.tiff', 'label': 0}
{'filename': '1960_10923.tiff', 

{'filename': '1979_11092.tiff', 'label': 0}
{'filename': '1979_11093.tiff', 'label': 0}
{'filename': '1979_11095.tiff', 'label': 0}
{'filename': '1979_11094.tiff', 'label': 0}
{'filename': '1979_11096.tiff', 'label': 0}
{'filename': '1979_11097.tiff', 'label': 0}
{'filename': '1980_11098.tiff', 'label': 0}
{'filename': '1980_11100.tiff', 'label': 0}
{'filename': '1980_11099.tiff', 'label': 0}
{'filename': '1980_11101.tiff', 'label': 0}
{'filename': '1980_11102.tiff', 'label': 0}
{'filename': '1980_11103.tiff', 'label': 0}
{'filename': '1980_11104.tiff', 'label': 0}
{'filename': '1980_11105.tiff', 'label': 0}
